In [56]:
import os
import re

import pandas as pd
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import FactorAnalysis, TruncatedSVD


In [19]:
DATA_PATH = 'data'

df = pd.read_json(os.path.join(DATA_PATH, 'followers_2016_rehydrated.jsonl'),
                            lines=True
                            ).set_index('id')

In [20]:
df.head()

,profile_image_url,username,public_metrics,description,protected,created_at,verified,name,pinned_tweet_id,entities,location,url,withheld
id,,,,,,,,,,,,,
1596715669684363264,https://pbs.twimg.com/profile_images/160820000...,Alyssa12348597,"{'followers_count': 149, 'following_count': 35...","Like fitness do yoga, Play golf for fun not pr...",False,2022-11-27 04:01:08+00:00,False,Beautiful flower,NaN,NaN,NaN,NaN,NaN
1599548681694466048,https://pbs.twimg.com/profile_images/159954889...,ElisaFenlon,"{'followers_count': 3, 'following_count': 304,...",,False,2022-12-04 23:38:49+00:00,False,Elisa Fenlon,NaN,NaN,NaN,NaN,NaN
1557880267653533696,https://pbs.twimg.com/profile_images/155788042...,AleciaDelbene,"{'followers_count': 37, 'following_count': 169...",🙊Trampoline🧡Artist,False,2022-08-12 00:03:04+00:00,False,Alecia Delbene,NaN,NaN,NaN,NaN,NaN
1559378560384507904,https://pbs.twimg.com/profile_images/155937871...,CharleighPotti1,"{'followers_count': 33, 'following_count': 163...",,False,2022-08-16 03:16:45+00:00,False,Charleigh Pottier,NaN,NaN,NaN,NaN,NaN
1588616089159843840,https://pbs.twimg.com/profile_images/158861629...,CatherineSeagr8,"{'followers_count': 12, 'following_count': 703...",,False,2022-11-04 19:36:17+00:00,False,Catherine Seagroves,NaN,NaN,NaN,NaN,NaN


In [21]:
df[~df.description.isna()]

,profile_image_url,username,public_metrics,description,protected,created_at,verified,name,pinned_tweet_id,entities,location,url,withheld
id,,,,,,,,,,,,,
1596715669684363264,https://pbs.twimg.com/profile_images/160820000...,Alyssa12348597,"{'followers_count': 149, 'following_count': 35...","Like fitness do yoga, Play golf for fun not pr...",False,2022-11-27 04:01:08+00:00,False,Beautiful flower,NaN,NaN,NaN,NaN,NaN
1599548681694466048,https://pbs.twimg.com/profile_images/159954889...,ElisaFenlon,"{'followers_count': 3, 'following_count': 304,...",,False,2022-12-04 23:38:49+00:00,False,Elisa Fenlon,NaN,NaN,NaN,NaN,NaN
1557880267653533696,https://pbs.twimg.com/profile_images/155788042...,AleciaDelbene,"{'followers_count': 37, 'following_count': 169...",🙊Trampoline🧡Artist,False,2022-08-12 00:03:04+00:00,False,Alecia Delbene,NaN,NaN,NaN,NaN,NaN
1559378560384507904,https://pbs.twimg.com/profile_images/155937871...,CharleighPotti1,"{'followers_count': 33, 'following_count': 163...",,False,2022-08-16 03:16:45+00:00,False,Charleigh Pottier,NaN,NaN,NaN,NaN,NaN
1588616089159843840,https://pbs.twimg.com/profile_images/158861629...,CatherineSeagr8,"{'followers_count': 12, 'following_count': 703...",,False,2022-11-04 19:36:17+00:00,False,Catherine Seagroves,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
824443214937497600,https://pbs.twimg.com/profile_images/857030758...,BLISSMagAlberta,"{'followers_count': 462, 'following_count': 13...",Attain total BLISS - articles featuring #beer ...,False,2017-01-26 02:25:54+00:00,False,BLISS Magazine,8.877268e+17,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...","Alberta, Canada",https://t.co/Xfthprl0Ky,NaN
857648235585740800,https://pbs.twimg.com/profile_images/857651838...,Mizzboogz,"{'followers_count': 29, 'following_count': 67,...","Passionate about life and love , lgbt and prou...",False,2017-04-27 17:30:48+00:00,False,Ashley Salazar,NaN,"{'description': {'mentions': [{'start': 80, 'e...","High River, Alberta",NaN,NaN
2767912537,https://pbs.twimg.com/profile_images/781211374...,BigAlsSmoke,"{'followers_count': 23717, 'following_count': ...",Big Al's Smoke & Gifts is the #1 premier smoke...,False,2014-08-25 20:56:23+00:00,False,Big Al's Smoke,NaN,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...","Berkeley, CA",https://t.co/Xeb2pE1Luq,NaN


In [54]:
remove_urls = lambda stringliteral: re.sub(r'http\S+', '', stringliteral)

In [57]:
descs = df.description.dropna().apply(remove_urls).str.strip()

In [58]:
descs = descs[descs.str.len()>0]

In [59]:
vec = HashingVectorizer(stop_words='english', n_features=10**4)

In [60]:
descs_vec = vec.fit_transform(descs)

In [61]:
descs_vec

<29449x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 240381 stored elements in Compressed Sparse Row format>

In [68]:
svd = TruncatedSVD(n_components=3)

In [69]:
descs_svd = svd.fit_transform(descs_vec)

In [72]:
descs_df = pd.DataFrame(descs_svd, columns=['component1', 'component2', 'component3'], index=descs.index)

In [73]:
descs_df['desc'] = descs

In [74]:
descs_df.loc[descs_df.component2.sort_values(ascending=False).index, ['component1', 'desc']]

,component1,desc
id,,
1041659562221162496,0.352973,LIFE👌
169189767,0.352973,MY LIFE
1563861351117774848,0.352973,Everything in life…
3668114774,0.328663,"Life Is Life, God is Great 👌 👌"
706366726682705920,0.290830,A life without cause is a life without effect
...,...,...
1191761651017691136,0.855076,I love LOVE
742642138177933312,0.855076,I love you
490148865,0.855076,♥ Love


In [76]:
descs_df.loc[descs_df.component1.sort_values(ascending=False).index, ['component2', 'desc']]


,component2,desc
id,,
177683234,-0.215536,"I love life, I love music,I love food,I love B..."
755967018717720576,-0.461123,I Love me.
1572628003753967616,-0.461123,LOVE💚🙏
1701243529,-0.461123,"I DO, I DO, I DO,, I DO IT FOR YOU!... LOVE YO..."
4530295034,-0.461123,love yourself more than they ever could.
...,...,...
3078559381,0.145249,God loving and Pink Floyd. Conservative belief...
759084188029030400,0.135100,@dohertyshannen my amazing and beautiful idol ...
167488086,0.126769,"soy un amor , inténtalo una y mil veces mas ha..."


In [77]:
descs_df.loc[descs_df.component1.sort_values(ascending=False).index, ['component3', 'desc']]

,component3,desc
id,,
177683234,0.153722,"I love life, I love music,I love food,I love B..."
755967018717720576,0.127824,I Love me.
1572628003753967616,0.127824,LOVE💚🙏
1701243529,0.127824,"I DO, I DO, I DO,, I DO IT FOR YOU!... LOVE YO..."
4530295034,0.127824,love yourself more than they ever could.
...,...,...
3078559381,-0.069309,God loving and Pink Floyd. Conservative belief...
759084188029030400,-0.052998,@dohertyshannen my amazing and beautiful idol ...
167488086,-0.042409,"soy un amor , inténtalo una y mil veces mas ha..."
